In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import textblob
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
import pickle

In [2]:
def export_model(model, file_name):
    #serializing our model to a file called {file_name}.pkl
    pickle.dump(model, open('models/' + file_name + '.pkl', 'wb'))

In [3]:
def report_to_csv(report_dict, output_file_name):
    df = pd.DataFrame(report_dict).transpose()
    df.to_csv('results/' + output_file_name + '.csv', encoding='utf-8')

In [4]:
def fit_predict_evaluate(pipeline, X_train, X_test, y_train, y_test, description):
    print(description)
    
    # Fitting
    pipeline.fit(X_train, y_train)
    
    # Export model
    export_model(pipeline, file_name=description)
    
    # Predict
    y_pred = pipeline.predict(X_test)
    
    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy)
    print()
    
    # Export to .csv
    report = classification_report(y_test, y_pred, output_dict=True)
    report['accuracy'] = {' ': accuracy}
    report_to_csv(report, description)
    
    return accuracy

In [5]:
def model_cv(splits, X, y, pipeline, description, category):
    print(description)
    
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
    reports = []
    accuracy = []
    
    for train, test in kfold.split(X, y):
        model_fit = pipeline.fit(X.iloc[train], y.iloc[train])
        prediction = model_fit.predict(X.iloc[test])
        accuracy.append(accuracy_score(y.iloc[test], prediction))
        reports.append(classification_report(y.iloc[test], prediction, output_dict=True))
    
    if (category == 'Bug'):
        report_avg_dict = bug_report_avg(reports)
    elif (category == 'Feature'):
        report_avg_dict = feature_report_avg(reports)
    elif (category == 'UserExperience'):
        report_avg_dict = ux_report_avg(reports)
    elif (category == 'Rating'):
        report_avg_dict = rating_report_avg(reports)
    else:
        raise ValueError('Category must be Bug, Feature, UX, or Rating.')
    
    accuracy = np.mean(accuracy)
    print('Accuracy:', accuracy)
    print()
    
    # Export to .csv
    report_avg_dict['accuracy'] = {' ': accuracy}
    report_to_csv(report_avg_dict, description)
    
    return accuracy

In [6]:
def bug_report_avg(reports):
    df = pd.DataFrame.from_dict(reports)
    
    bug_arr = df['Bug'].values 
    not_bug_arr = df['Not Bug'].values

    macro_arr = df['macro avg'].values 
    micro_arr = df['micro avg'].values 
    weighted_arr = df['weighted avg'].values
    
    # lambda functions
    precision_lambda = lambda x: x['precision']
    recall_lambda = lambda x: x['recall']
    f1_lambda = lambda x: x['f1-score']
    support_lambda = lambda x: x['support']
    
    # Mean function
    def my_mean(function, array):
        return np.mean(np.array(list(map(function, array))))
    
    # Bug
    bugs_precision_avg = my_mean(precision_lambda, bug_arr)
    bugs_recall_avg = my_mean(recall_lambda, bug_arr)
    bugs_f1_avg = my_mean(f1_lambda, bug_arr)
    bugs_support_avg = my_mean(support_lambda, bug_arr)
    
    # Not Bug
    not_bugs_precision_avg = my_mean(precision_lambda, not_bug_arr)
    not_bugs_recall_avg = my_mean(recall_lambda, not_bug_arr)
    not_bugs_f1_avg = my_mean(f1_lambda, not_bug_arr)
    not_bugs_support_avg = my_mean(support_lambda, not_bug_arr)
    
    # Macro
    macro_precision_avg = my_mean(precision_lambda, macro_arr)
    macro_recall_avg = my_mean(recall_lambda, macro_arr)
    macro_f1_avg = my_mean(f1_lambda, macro_arr)
    macro_support_avg = my_mean(support_lambda, macro_arr)
    
    # Micro
    micro_precision_avg = my_mean(precision_lambda, micro_arr)
    micro_recall_avg = my_mean(recall_lambda, micro_arr)
    micro_f1_avg = my_mean(f1_lambda, micro_arr)
    micro_support_avg = my_mean(support_lambda, micro_arr)
    
    # Weight
    weights_precision_avg = my_mean(precision_lambda, weighted_arr)
    weights_recall_avg = my_mean(recall_lambda, weighted_arr)
    weights_f1_avg = my_mean(f1_lambda, weighted_arr)
    weights_support_avg = my_mean(support_lambda, weighted_arr)
    
    # Aggregate function
    def my_dict(precision_avg, recall_avg, f1_avg, support_avg):
        return {"precision": precision_avg, 
               "recall": recall_avg, 
               "f1-score": f1_avg, 
               "support": support_avg}
    
    # Combine data then return
    return {"Bug": my_dict(bugs_precision_avg, bugs_recall_avg, bugs_f1_avg, bugs_support_avg),
            "Not Bug": my_dict(not_bugs_precision_avg, not_bugs_recall_avg, not_bugs_f1_avg, not_bugs_support_avg), 
            "micro avg": my_dict(micro_precision_avg, micro_recall_avg, micro_f1_avg, micro_support_avg), 
            "macro avg": my_dict(macro_precision_avg, macro_recall_avg, macro_f1_avg, macro_support_avg), 
            "weighted avg": my_dict(weights_precision_avg, weights_recall_avg, weights_f1_avg, weights_support_avg)}

In [7]:
def feature_report_avg(reports):
    df = pd.DataFrame.from_dict(reports)
    
    feature_arr = df['Feature'].values
    not_feature_arr = df['Not Feature'].values

    macro_arr = df['macro avg'].values 
    micro_arr = df['micro avg'].values 
    weighted_arr = df['weighted avg'].values
    
    # lambda functions
    precision_lambda = lambda x: x['precision']
    recall_lambda = lambda x: x['recall']
    f1_lambda = lambda x: x['f1-score']
    support_lambda = lambda x: x['support']
    
    # Mean function
    def my_mean(function, array):
        return np.mean(np.array(list(map(function, array))))
    
    # Feature
    features_precision_avg = my_mean(precision_lambda, feature_arr)
    features_recall_avg = my_mean(recall_lambda, feature_arr)
    features_f1_avg = my_mean(f1_lambda, feature_arr)
    features_support_avg = my_mean(support_lambda, feature_arr)
    
    # Not Feature
    not_features_precision_avg = my_mean(precision_lambda, not_feature_arr)
    not_features_recall_avg = my_mean(recall_lambda, not_feature_arr)
    not_features_f1_avg = my_mean(f1_lambda, not_feature_arr)
    not_features_support_avg = my_mean(support_lambda, not_feature_arr)
    
    # Macro
    macro_precision_avg = my_mean(precision_lambda, macro_arr)
    macro_recall_avg = my_mean(recall_lambda, macro_arr)
    macro_f1_avg = my_mean(f1_lambda, macro_arr)
    macro_support_avg = my_mean(support_lambda, macro_arr)
    
    # Micro
    micro_precision_avg = my_mean(precision_lambda, micro_arr)
    micro_recall_avg = my_mean(recall_lambda, micro_arr)
    micro_f1_avg = my_mean(f1_lambda, micro_arr)
    micro_support_avg = my_mean(support_lambda, micro_arr)
    
    # Weight
    weights_precision_avg = my_mean(precision_lambda, weighted_arr)
    weights_recall_avg = my_mean(recall_lambda, weighted_arr)
    weights_f1_avg = my_mean(f1_lambda, weighted_arr)
    weights_support_avg = my_mean(support_lambda, weighted_arr)
    
    # Aggregate function
    def my_dict(precision_avg, recall_avg, f1_avg, support_avg):
        return {"precision": precision_avg, 
               "recall": recall_avg, 
               "f1-score": f1_avg, 
               "support": support_avg}
    
    # Combine data then return
    return {"Feature": my_dict(features_precision_avg, features_recall_avg, features_f1_avg, features_support_avg),
            "Not Feature": my_dict(not_features_precision_avg, not_features_recall_avg, not_features_f1_avg, not_features_support_avg),
            "micro avg": my_dict(micro_precision_avg, micro_recall_avg, micro_f1_avg, micro_support_avg), 
            "macro avg": my_dict(macro_precision_avg, macro_recall_avg, macro_f1_avg, macro_support_avg), 
            "weighted avg": my_dict(weights_precision_avg, weights_recall_avg, weights_f1_avg, weights_support_avg)}

In [8]:
def ux_report_avg(reports):
    df = pd.DataFrame.from_dict(reports)
    
    ux_arr = df['UserExperience'].values
    not_ux_arr = df['Not UserExperience'].values

    macro_arr = df['macro avg'].values 
    micro_arr = df['micro avg'].values 
    weighted_arr = df['weighted avg'].values
    
    # lambda functions
    precision_lambda = lambda x: x['precision']
    recall_lambda = lambda x: x['recall']
    f1_lambda = lambda x: x['f1-score']
    support_lambda = lambda x: x['support']
    
    # Mean function
    def my_mean(function, array):
        return np.mean(np.array(list(map(function, array))))
    
    # UX
    ux_precision_avg = my_mean(precision_lambda, ux_arr)
    ux_recall_avg = my_mean(recall_lambda, ux_arr)
    ux_f1_avg = my_mean(f1_lambda, ux_arr)
    ux_support_avg = my_mean(support_lambda, ux_arr)
    
    # Not UX
    not_ux_precision_avg = my_mean(precision_lambda, not_ux_arr)
    not_ux_recall_avg = my_mean(recall_lambda, not_ux_arr)
    not_ux_f1_avg = my_mean(f1_lambda, not_ux_arr)
    not_ux_support_avg = my_mean(support_lambda, not_ux_arr)
    
    # Macro
    macro_precision_avg = my_mean(precision_lambda, macro_arr)
    macro_recall_avg = my_mean(recall_lambda, macro_arr)
    macro_f1_avg = my_mean(f1_lambda, macro_arr)
    macro_support_avg = my_mean(support_lambda, macro_arr)
    
    # Micro
    micro_precision_avg = my_mean(precision_lambda, micro_arr)
    micro_recall_avg = my_mean(recall_lambda, micro_arr)
    micro_f1_avg = my_mean(f1_lambda, micro_arr)
    micro_support_avg = my_mean(support_lambda, micro_arr)
    
    # Weight
    weights_precision_avg = my_mean(precision_lambda, weighted_arr)
    weights_recall_avg = my_mean(recall_lambda, weighted_arr)
    weights_f1_avg = my_mean(f1_lambda, weighted_arr)
    weights_support_avg = my_mean(support_lambda, weighted_arr)
    
    # Aggregate function
    def my_dict(precision_avg, recall_avg, f1_avg, support_avg):
        return {"precision": precision_avg, 
               "recall": recall_avg, 
               "f1-score": f1_avg, 
               "support": support_avg}
    
    # Combine data then return
    return {"UserExperience": my_dict(ux_precision_avg, ux_recall_avg, ux_f1_avg, ux_support_avg),
            "Not UserExperience": my_dict(not_ux_precision_avg, not_ux_recall_avg, not_ux_f1_avg, not_ux_support_avg),
            "micro avg": my_dict(micro_precision_avg, micro_recall_avg, micro_f1_avg, micro_support_avg), 
            "macro avg": my_dict(macro_precision_avg, macro_recall_avg, macro_f1_avg, macro_support_avg), 
            "weighted avg": my_dict(weights_precision_avg, weights_recall_avg, weights_f1_avg, weights_support_avg)}

In [9]:
def rating_report_avg(reports):
    df = pd.DataFrame.from_dict(reports)
     
    rating_arr = df['Rating'].values
    not_rating_arr = df['Not Rating'].values

    macro_arr = df['macro avg'].values 
    micro_arr = df['micro avg'].values 
    weighted_arr = df['weighted avg'].values
    
    # lambda functions
    precision_lambda = lambda x: x['precision']
    recall_lambda = lambda x: x['recall']
    f1_lambda = lambda x: x['f1-score']
    support_lambda = lambda x: x['support']
    
    # Mean function
    def my_mean(function, array):
        return np.mean(np.array(list(map(function, array))))
    
    # Rating
    ratings_precision_avg = my_mean(precision_lambda, rating_arr)
    ratings_recall_avg = my_mean(recall_lambda, rating_arr)
    ratings_f1_avg = my_mean(f1_lambda, rating_arr)
    ratings_support_avg = my_mean(support_lambda, rating_arr)
    
    # Not Rating
    not_ratings_precision_avg = my_mean(precision_lambda, not_rating_arr)
    not_ratings_recall_avg = my_mean(recall_lambda, not_rating_arr)
    not_ratings_f1_avg = my_mean(f1_lambda, not_rating_arr)
    not_ratings_support_avg = my_mean(support_lambda, not_rating_arr)
    
    # Macro
    macro_precision_avg = my_mean(precision_lambda, macro_arr)
    macro_recall_avg = my_mean(recall_lambda, macro_arr)
    macro_f1_avg = my_mean(f1_lambda, macro_arr)
    macro_support_avg = my_mean(support_lambda, macro_arr)
    
    # Micro
    micro_precision_avg = my_mean(precision_lambda, micro_arr)
    micro_recall_avg = my_mean(recall_lambda, micro_arr)
    micro_f1_avg = my_mean(f1_lambda, micro_arr)
    micro_support_avg = my_mean(support_lambda, micro_arr)
    
    # Weight
    weights_precision_avg = my_mean(precision_lambda, weighted_arr)
    weights_recall_avg = my_mean(recall_lambda, weighted_arr)
    weights_f1_avg = my_mean(f1_lambda, weighted_arr)
    weights_support_avg = my_mean(support_lambda, weighted_arr)
    
    # Aggregate function
    def my_dict(precision_avg, recall_avg, f1_avg, support_avg):
        return {"precision": precision_avg, 
               "recall": recall_avg, 
               "f1-score": f1_avg, 
               "support": support_avg}
    
    # Combine data then return
    return {"Rating": my_dict(ratings_precision_avg, ratings_recall_avg, ratings_f1_avg, ratings_support_avg),
            "Not Rating": my_dict(not_ratings_precision_avg, not_ratings_recall_avg, not_ratings_f1_avg, not_ratings_support_avg),
            "micro avg": my_dict(micro_precision_avg, micro_recall_avg, micro_f1_avg, micro_support_avg), 
            "macro avg": my_dict(macro_precision_avg, macro_recall_avg, macro_f1_avg, macro_support_avg), 
            "weighted avg": my_dict(weights_precision_avg, weights_recall_avg, weights_f1_avg, weights_support_avg)}

In [10]:
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

In [11]:
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [12]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [13]:
# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    pos_family = {
                    'noun': ['NN', 'NNS', 'NNP', 'NNPS'],
                    'pron': ['PRP', 'PRP$', 'WP', 'WP$'],
                    'verb': ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
                    'adj':  ['JJ', 'JJR', 'JJS'],
                    'adv': ['RB', 'RBR', 'RBS', 'WRB']
                }
    
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

In [14]:
# Word2Vec

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec, dim):
        self.word2vec = word2vec
        if len(word2vec) > 0:
            self.dim = dim
        else:
            self.dim = 0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    
# and a tf-idf version of the same
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec, dim):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec) > 0:
            self.dim = dim
        else:
            self.dim = 0
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])